# 3일차 2교시 기본 연산 다루기

> 스파크의 "기본 연산자" 와 "데이터프레임"에 대해 학습합니다

## 1. 기본 연산자
---
### 1-1. 데이터 프레임 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| df.printSchema() | 스키마 정보를 출력합니다. | - |
| df.schema | StructType 스키마를 반환합니다 | - |
| df.columns | 컬럼명 정보를 반환합니다 | - |
| df.show(n) | 데이터 n 개를 출력합니다 | - |
| df.first() | 데이터 프레임의 첫 번째 Row 를 반환합니다 | - |
| df.head(n) | 데이터 프레임의 처음부터 n 개의 Row 를 반환합니다 | - |
| df.createOrReplaceTempView | 임시 뷰 테이블을 생성합니다 | - |
| df.union(newdf) | 데이터프레임 간의 유니온 연산을 수행합니다 | - |
| df.limit(n) | 추출할 로우수 제한 | T |
| df.repartition(n) | 파티션 재분배, 셔플발생 | - |
| df.coalesce() | 셔플하지 않고 파티션을 병합 | 마지막 스테이지의 reduce 수가 줄어드는 효과로 성능저하에 유의해야 합니다 |
| df.collect() | 모든 데이터 수집, 반환 | A |
| df.take(n) | 상위 n개 로우 반환 | A |

---
### 1-2. 컬럼 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| df.select | 컬럼이나 표현식 사용  | - |
| df.selectExpr | 문자열 표현식 사용 = df.select(expr()) | - |
| df.withColumn(컬럼명, 표현식) | 컬럼 추가, 비교, 컬럼명 변경 | - |
| df.withColumnRenamed(old_name, new_name) | 컬럼명 변경 | - |
| df.drop() | 컬럼 삭제 | - |
| df.where | 로우 필터링 | - |
| df.filter | 로우 필터링 | - |
| df.sort, df.orderBy | 정렬 | - |
| df.sortWithinPartitions | 파티션별 정렬 | - |

---
### 1-3. 기타 함수
| 함수 | 설명 | 기타 |
| - | - | - |
| expr("someCol - 5") | 표현식 | - |
| lit() | 리터럴 | - |
| cast() | 컬럼 데이터 타입 변경 | - |
| distinct() | unique row | - |
| desc(), asc() | 정렬 순서 | - |


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

## 2 스파크 데이터 다루기

### 2.1 파일로 부터 테이블 만들어 사용하기

In [2]:
print("# 원시 데이터로 부터 읽거나, Spark SQL 통한 결과는 항상 데이터프레임이 생성됩니다")
df = spark.read.json("data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("2015_summary")

sql_result = spark.sql("SELECT * FROM 2015_summary").show(5)

# 원시 데이터로 부터 읽거나, Spark SQL 통한 결과는 항상 데이터프레임이 생성됩니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



### 2.2 특정 컬럼 선택 (select, selectExpr)
> 아래의 모든 예제에서 컬럼 선택 시에 select(col("컬럼명")) 으로 접근할 수도 있지만 **selectExpr("컬럼명") 이 간결하기 때문에 앞으로는 가능한 표현식으로 사용**하겠습니다 <br>
컬럼 표현식의 경우 반드시 하나의 컬럼은 하나씩 표현되어야만 합니다.  <br>
잘된예 : "컬럼1", "컬럼2" <br>
잘못된예: "컬럼1, 컬럼2"

In [3]:
from pyspark.sql.functions import *

print("# select 표현은 컬럼만 입력이 가능하며, 함수나 기타 표현식을 사용할 수 없습니다. 사용하기 위해서는 functions 를 임포트 하고, 개별 함수의 특징을 잘 이해하고 사용해야 합니다")
df.select(upper(col("DEST_COUNTRY_NAME")), "ORIGIN_COUNTRY_NAME").show(2)

print("# selectExpr 별도의 임포트 없이, 모든 표현식을 사용할 수 있습니다")
df.selectExpr("upper(DEST_COUNTRY_NAME)", "ORIGIN_COUNTRY_NAME").show(2)

# select 표현은 컬럼만 입력이 가능하며, 함수나 기타 표현식을 사용할 수 없습니다. 사용하기 위해서는 functions 를 임포트 하고, 개별 함수의 특징을 잘 이해하고 사용해야 합니다
+------------------------+-------------------+
|upper(DEST_COUNTRY_NAME)|ORIGIN_COUNTRY_NAME|
+------------------------+-------------------+
|           UNITED STATES|            Romania|
|           UNITED STATES|            Croatia|
+------------------------+-------------------+
only showing top 2 rows

# selectExpr 별도의 임포트 없이, 모든 표현식을 사용할 수 있습니다
+------------------------+-------------------+
|upper(DEST_COUNTRY_NAME)|ORIGIN_COUNTRY_NAME|
+------------------------+-------------------+
|           UNITED STATES|            Romania|
|           UNITED STATES|            Croatia|
+------------------------+-------------------+
only showing top 2 rows



In [4]:
print("# 컬럼의 앨리어스 혹은 전체 컬럼을 위한 * 도 사용할 수 있습니다")
df.selectExpr("DEST_COUNTRY_NAME as newColmnName", "DEST_COUNTRY_NAME").show(2)

df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry").show(2)

# 컬럼의 앨리어스 혹은 전체 컬럼을 위한 * 도 사용할 수 있습니다
+-------------+-----------------+
| newColmnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



### 2.3 상수값 사용하기

In [5]:
# 리터럴(literal)을 사용한 리터럴 상수 값 컬럼 추가
from pyspark.sql.functions import lit

# df.select(expr("*"), lit(1).alias("One")).show(2)
df.selectExpr("*", "1 as One").show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### 2.4 컬럼 추가하기

In [6]:
print("# withColumn(컬럼명, 표현식) 으로 컬럼 추가")
df.withColumn("numberOne", lit(1)).show(2)

# withColumn(컬럼명, 표현식) 으로 컬럼 추가
+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [7]:
print("# 컬럼의 대소 비교를 통한 불리언 값 반환")
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

# 컬럼의 대소 비교를 통한 불리언 값 반환
+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [8]:
print("# 존재하는 컬럼을 표현식을 통해 새로운 컬럼 생성, 기존 컬럼을 삭제")
before = df
before.printSchema()

after = before.withColumn("Destination", expr("DEST_COUNTRY_NAME"))
after.printSchema()

# 존재하는 컬럼을 표현식을 통해 새로운 컬럼 생성, 기존 컬럼을 삭제
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- Destination: string (nullable = true)



### 2.5 컬럼명 바꾸기

In [9]:
print("# 컬럼 명 변경하기")
df.withColumnRenamed("DEST_COUNTRY_NAME", "Destination").columns

# 컬럼 명 변경하기


['Destination', 'ORIGIN_COUNTRY_NAME', 'count']

### 2.6 컬럼 제거하기

In [10]:
print("# 특정 컬럼을 제거합니다")
df.printSchema()
df.drop("ORIGIN_COUNTRY_NAME").columns

# 특정 컬럼을 제거합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



['DEST_COUNTRY_NAME', 'count']

In [11]:
print("# 기본적으로 스파크는 대소문자를 가리지 않지만, 옵션을 통해서 구분이 가능합니다")
spark.conf.set('spark.sql.caseSensitive', True)
caseSensitive = df.drop("dest_country_name")
caseSensitive.printSchema()

spark.conf.set('spark.sql.caseSensitive', False)
caseInsensitive = df.drop("dest_country_name")
caseInsensitive.printSchema()

# 기본적으로 스파크는 대소문자를 가리지 않지만, 옵션을 통해서 구분이 가능합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [12]:
print("# 한 번에 여러 컬럼도 삭제할 수 있습니다")
df.printSchema()
df.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns # 여러 컬럼을 지우기

# 한 번에 여러 컬럼도 삭제할 수 있습니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



['count']

### 2.7 컬럼의 데이터 타입 변경하기

In [13]:
print("# 컬럼의 데이터 유형을 변경합니다")
df.printSchema()

int2str = df.withColumn("str_count", col("count").cast("string"))
int2str.show(5)
int2str.printSchema()

str2int = int2str.withColumn("int_count", col("str_count").cast("int"))
str2int.show(5)
str2int.printSchema()

# 컬럼의 데이터 유형을 변경합니다
root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|       15|
|    United States|            Croatia|    1|        1|
|    United States|            Ireland|  344|      344|
|            Egypt|      United States|   15|       15|
|    United States|              India|   62|       62|
+-----------------+-------------------+-----+---------+
only showing top 5 rows

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- str_count: string (nullable = true)

+-----------------+-------------------+-----+---------+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|str_count|int_cou

### 2.8 레코드 필터링

In [14]:
print("# Where 와 Filter 는 동일합니다")
df.where("count < 2").show(2)
df.filter("count < 2").show(2)

print("# 같은 표현식에 여러 필터를 적용하는 것도 가능합니다")
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)

# Where 와 Filter 는 동일합니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

# 같은 표현식에 여러 필터를 적용하는 것도 가능합니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### 2.9 유일 값 (DISTINCT)

In [15]:
""" distinct 함수 """
print(df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count())
print(df.select("ORIGIN_COUNTRY_NAME").distinct().count())
# distinctcount?

256
125


### 2.10 정렬

In [16]:
print("# sort 와 orderBy 함수는 동일한 효과를 가집니다")
df.sort("count").show(2)
df.orderBy("count", "DEST_COUNTRY_NAME").show(2)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(2)

# sort 와 orderBy 함수는 동일한 효과를 가집니다
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [17]:
from pyspark.sql.functions import *
print("# asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드로 null의 정렬 순서를 지정")
df.sort("DEST_COUNTRY_NAME").show(1)
df.sort(df["DEST_COUNTRY_NAME"].asc_nulls_first()).show(1)
df.sort(df.DEST_COUNTRY_NAME.asc_nulls_first()).show(1)

# asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드로 null의 정렬 순서를 지정
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
+-----------------+-------------------+-----+
only showing top 1 row



In [18]:
print("# 정렬의 경우 예약어 컬럼명에 유의해야 하므로, expr 을 사용하거나, 명시적으로 구조화 API 를 사용하는 것도 좋습니다") 
from pyspark.sql.functions import desc, asc
df.orderBy(df["count"].desc()).show(2)
df.orderBy(df.ORIGIN_COUNTRY_NAME.desc(), df.DEST_COUNTRY_NAME.asc()).show(2)
df.orderBy(expr("ORIGIN_COUNTRY_NAME DESC"), expr("DEST_COUNTRY_NAME ASC")).show(2)

# 정렬의 경우 예약어 컬럼명에 유의해야 하므로, expr 을 사용하거나, 명시적으로 구조화 API 를 사용하는 것도 좋습니다
+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Vietnam|    2|
|    United States|          Venezuela|  246|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|             Angola|   13|
|    United States|           Anguilla|   38|
+-----------------+-------------------+-----+
only showing top 2 rows



### 2.11 로우 수 제한 (LIMIT)

In [19]:
df.limit(5).show()
df.orderBy(expr("count desc")).limit(6).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



### 실습#3 data/tbl_user.csv 파일을 읽고, 가장 최근에 가입(u_signup)한 5명을 출력하세요
> 참고: sort, desc, from pyspark.sql.functions import *

In [20]:
from pyspark.sql.functions import *

user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")
user.createOrReplaceTempView("user")
spark.sql("select * from user order by u_signup desc limit 5").show()

+----+----------+--------+--------+
|u_id|    u_name|u_gender|u_signup|
+----+----------+--------+--------+
|   9|  최컴퓨터|      남|20201124|
|   8|  조노트북|      여|20161201|
|   7|  임모바일|      남|20040807|
|   6|  윤디오스|      남|20040101|
|   5|유코드제로|      여|20021029|
+----+----------+--------+--------+



### 실습#4 data/tbl_purchase.csv 파일을 읽고, 200만원 (p_amount) 이상 구매한 이용자 목록을 출력하세요
> 참고: filter

In [21]:
purchase = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_purchase.csv")
purchase.filter("p_amount > 2000000").show()

+----------+-----+----+-----------+--------+
|    p_time|p_uid|p_id|     p_name|p_amount|
+----------+-----+----+-----------+--------+
|1603674500|    4|2003|LG Computer| 4500000|
|1603665955|    5|2004|    LG Gram| 3500000|
|1603666155|    5|2004|      LG TV| 2500000|
+----------+-----+----+-----------+--------+

